<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/vector_CLIP%26YOLO%26Qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
!unzip '/content/drive/MyDrive/drug/data 50 class add_augment3.zip'

#Crop

In [ ]:
import os
from PIL import Image
from ultralytics import YOLO

# Load your YOLOv8 model
model = YOLO('/content/drive/MyDrive/drug/best_new.pt')

# Define the input path (can be a single image or a folder)
path = '/content/data 50 class add_augment'

# Define the crop directory
crop_dir = '/content/drive/MyDrive/drug/Corp_4800'
os.makedirs(crop_dir, exist_ok=True)

# Function to perform object detection and cropping
def detect_and_crop(image_path, save_path):
    # Predict with the YOLO model
    results = model(image_path)

    # Parse the results
    detected = False
    for result in results:
        # Extract boxes, labels, and scores
        boxes = result.boxes.xyxy  # x1, y1, x2, y2
        labels = result.boxes.cls  # class ids
        scores = result.boxes.conf  # confidence scores

        # Convert tensor to numpy for easier processing
        boxes = boxes.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        scores = scores.cpu().detach().numpy()

        # Threshold for detection confidence
        threshold = 0.3

        # List of target labels (Replace with actual indices of your classes in the YOLO model)
        target_labels = ['Amlopine_10mg', 'Amlopine_5mg', 'Anapril_5mg', 'Betalol-10-mg', 'Betalol-40-mg',
                         'Blopress-16-mg', 'Blopress-8-mg', 'Blopress-Plus-16-mg', 'Blopress-Plus-8-mg',
                         'Caduet-10_10-mg', 'Caduet-5_10-mg', 'Daonil-5-mg', 'Diamicron-MR-60-mg',
                         'Diovan-160-mg', 'Diovan-80-mg', 'Forxiga-10-mg', 'Galvus-50-mg', 'Galvus_Met_50_1000mg',
                         'Gliclazide_80mg', 'Gliparil-2-mg', 'Glucophage-500-mg', 'Glucophage-850-mg',
                         'Glucophage-XR-1000-mg', 'Glucophage-XR-750-mg', 'Glyxambi-25_5-mg', 'Janumet-50_1000-mg',
                         'Januvia-100-mg', 'Jardiance-10-mg', 'Jardiance-25-mg', 'Jardiance-Duo-12_5_1000-mg',
                         'Lanzaar-100-mg', 'Lercadip-20-mg', 'Madiplot-10-mg', 'Madiplot_20mg',
                         'Metoprolol_Stada_100mg', 'Micardis_40mg', 'Micardis_Plus_80_12_5mg', 'Minidiab_5mg',
                         'Novonorm_1mg', 'Novonorm_2mg', 'Oseni_25_15mg', 'Poli_uretic', 'Prenolol_100mg',
                         'Prenolol_25mg', 'Prenolol_50mg', 'Tanzaril_50mg', 'Utmos_15mg', 'Utmos_30mg',
                         'Xigduo_XR_10_1000mg', 'Zanidip_10mg']

        # Create a map from YOLO class index to target label
        class_map = {i: label for i, label in enumerate(target_labels)}

        # Check if any of the target labels are detected
        for i, label in enumerate(labels):
            if class_map[label] in target_labels and scores[i] >= threshold:
                score = scores[i]
                box = boxes[i]

                # Get the coordinates and dimensions of the detected object
                x1, y1, x2, y2 = box

                # Open the original image
                im = Image.open(image_path)

                # Crop the image using the detected coordinates
                im_crop = im.crop((x1, y1, x2, y2))

                # Convert image mode to RGB if it's RGBA
                if im_crop.mode == 'RGBA':
                    im_crop = im_crop.convert('RGB')

                # Save the cropped image in the original subdirectory structure
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                im_crop.save(save_path)
                print(f"Cropped image saved as '{save_path}'.")
                detected = True
                return  # Exit after saving the first detected crop

    if not detected:
        print(f"Can't detect any target objects in '{image_path}'.")

# Check if path is a file or directory
if os.path.isfile(path):
    save_path = os.path.join(crop_dir, os.path.relpath(path, start=path))
    detect_and_crop(path, save_path)
elif os.path.isdir(path):
    # Loop through all subdirectories and files in the directory
    for root, dirs, files in os.walk(path):
        for file_name in sorted(files):
            file_path = os.path.join(root, file_name)
            if os.path.isfile(file_path):
                save_path = os.path.join(crop_dir, os.path.relpath(file_path, start=path))
                detect_and_crop(file_path, save_path)
else:
    print(f"Invalid path: {path}")

#Corp&Segment

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO

# Load your YOLOv8 segmentation model
model = YOLO('/content/drive/MyDrive/drug/best_segment.pt')

# Define the input path (can be a single image or a folder)
path = '/content/data 50 class add_augment'

# Define the crop directory
crop_dir = '/content/drive/MyDrive/drug/Corp&Segment_4800'
os.makedirs(crop_dir, exist_ok=True)

# Function to perform object detection, cropping, and segmentation
def detect_crop_and_segment(image_path, save_path):
    # Predict with the YOLO model
    results = model(image_path)

    # Parse the results
    detected = False
    for result in results:
        # Extract boxes, labels, and masks
        boxes = result.boxes.xyxy  # x1, y1, x2, y2
        labels = result.boxes.cls  # class ids
        scores = result.boxes.conf  # confidence scores

        # Convert tensor to numpy for easier processing
        boxes = boxes.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        scores = scores.cpu().detach().numpy()

        # Check if masks exist
        if result.masks is not None:
            masks = result.masks.data  # segmentation masks
            masks = masks.cpu().detach().numpy()
        else:
            masks = None

        # Threshold for detection confidence
        threshold = 0.3

        # List of target labels (Replace with actual indices of your classes in the YOLO model)
        target_labels = ['Amlopine_10mg', 'Amlopine_5mg', 'Anapril_5mg', 'Betalol-10-mg', 'Betalol-40-mg',
                         'Blopress-16-mg', 'Blopress-8-mg', 'Blopress-Plus-16-mg', 'Blopress-Plus-8-mg',
                         'Caduet-10_10-mg', 'Caduet-5_10-mg', 'Daonil-5-mg', 'Diamicron-MR-60-mg',
                         'Diovan-160-mg', 'Diovan-80-mg', 'Forxiga-10-mg', 'Galvus-50-mg', 'Galvus_Met_50_1000mg',
                         'Gliclazide_80mg', 'Gliparil-2-mg', 'Glucophage-500-mg', 'Glucophage-850-mg',
                         'Glucophage-XR-1000-mg', 'Glucophage-XR-750-mg', 'Glyxambi-25_5-mg', 'Janumet-50_1000-mg',
                         'Januvia-100-mg', 'Jardiance-10-mg', 'Jardiance-25-mg', 'Jardiance-Duo-12_5_1000-mg',
                         'Lanzaar-100-mg', 'Lercadip-20-mg', 'Madiplot-10-mg', 'Madiplot_20mg',
                         'Metoprolol_Stada_100mg', 'Micardis_40mg', 'Micardis_Plus_80_12_5mg', 'Minidiab_5mg',
                         'Novonorm_1mg', 'Novonorm_2mg', 'Oseni_25_15mg', 'Poli_uretic', 'Prenolol_100mg',
                         'Prenolol_25mg', 'Prenolol_50mg', 'Tanzaril_50mg', 'Utmos_15mg', 'Utmos_30mg',
                         'Xigduo_XR_10_1000mg', 'Zanidip_10mg']

        # Create a map from YOLO class index to target label
        class_map = {i: label for i, label in enumerate(target_labels)}

        # Check if any of the target labels are detected
        for i, label in enumerate(labels):
            if class_map[label] in target_labels and scores[i] >= threshold:
                score = scores[i]
                box = boxes[i]

                # Get the coordinates and dimensions of the detected object
                x1, y1, x2, y2 = map(int, box)

                # Open the original image
                im = Image.open(image_path)
                im_np = np.array(im)

                # Crop the image using the detected coordinates
                im_crop = im_np[y1:y2, x1:x2]

                if masks is not None:
                    # Apply the segmentation mask to the cropped image
                    mask = masks[i]
                    mask_resized = cv2.resize(mask, (x2-x1, y2-y1), interpolation=cv2.INTER_NEAREST)
                    im_segmented = cv2.bitwise_and(im_crop, im_crop, mask=mask_resized.astype(np.uint8))
                else:
                    im_segmented = im_crop

                # Convert image back to PIL format for saving
                im_segmented_pil = Image.fromarray(im_segmented)

                # Save the segmented and cropped image in the original subdirectory structure
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                im_segmented_pil.save(save_path)
                print(f"Cropped and segmented image saved as '{save_path}'.")
                detected = True
                return  # Exit after saving the first detected crop

    if not detected:
        print(f"Can't detect any target objects in '{image_path}'.")

# Check if path is a file or directory
if os.path.isfile(path):
    save_path = os.path.join(crop_dir, os.path.relpath(path, start=path))
    detect_crop_and_segment(path, save_path)
elif os.path.isdir(path):
    # Loop through all subdirectories and files in the directory
    for root, dirs, files in os.walk(path):
        for file_name in sorted(files):
            file_path = os.path.join(root, file_name)
            if os.path.isfile(file_path):
                save_path = os.path.join(crop_dir, os.path.relpath(file_path, start=path))
                detect_crop_and_segment(file_path, save_path)
else:
    print(f"Invalid path: {path}")

#Qdrant

In [ ]:
!pip install tensorflow

In [ ]:
!pip install qdrant-client

In [ ]:
import os
import numpy as np
import torch
import uuid
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams
from transformers import CLIPProcessor, CLIPModel

# ใช้โมเดล CLIP ของ Hugging Face ในการทำ embedding
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# เชื่อมต่อกับ Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)

def create_collection(collection_name, vector_size, distance_metric):
    try:
        # ตรวจสอบว่าคอลเลกชันมีอยู่แล้วหรือไม่
        collections = qdrant_client.get_collections().collections
        existing_collections = [col.name for col in collections]

        if collection_name in existing_collections:
            print(f"Collection {collection_name} already exists.")
        else:
            # สร้างคอลเลกชัน
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=vector_size,  # ขนาดของเวกเตอร์ embedding
                    distance=distance_metric  # ระยะทางที่ใช้ในการคำนวณ
                )
            )
            print(f"Collection {collection_name} created successfully.")
    except Exception as e:
        print(f"An error occurred while creating the collection: {e}")

def image_embedding(path):
    img = Image.open(path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt", size=(640, 640))
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    embedding = outputs.cpu().numpy().flatten()
    return embedding

def process_images_in_folder(folder_path, collection_name):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedding = image_embedding(image_path)
        class_name = os.path.basename(folder_path)  # ใช้ชื่อโฟลเดอร์เป็นคลาส
        point_id = str(uuid.uuid4())  # ใช้ UUID เป็น ID
        point = PointStruct(id=point_id, vector=embedding.tolist(), payload={"class": class_name, "image_label": image_file})
        try:
            qdrant_client.upsert(collection_name=collection_name, points=[point])
            print(f"Uploaded {image_file} to collection {collection_name} with class {class_name}")
        except Exception as e:
            print(f"An error occurred while uploading {image_file}: {e}")

def process_all_folders(base_folder_path):
    collection_name = "vector_CLIP"  # ใช้ชื่อคอลเลกชันเดียวสำหรับเก็บข้อมูลทั้งหมด

    create_collection(collection_name, vector_size=768, distance_metric='Cosine')  # ขนาดเวกเตอร์ของ CLIPModel คือ 768

    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    for subfolder in subfolders:
        process_images_in_folder(subfolder, collection_name)
        print(f"Embedding data from {os.path.basename(subfolder)} has been uploaded to Qdrant")

# ตัวอย่างการใช้งาน
base_folder_path = '/content/drive/MyDrive/drug/Corp&Segment_4800'
process_all_folders(base_folder_path)

In [ ]:
import os
import numpy as np
import torch
import uuid
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams
from transformers import CLIPProcessor, CLIPModel

# ใช้โมเดล CLIP ของ Hugging Face ในการทำ embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # ตรวจสอบการใช้ GPU
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)  # ส่งโมเดลไปยัง GPU
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# เชื่อมต่อกับ Qdrant
qdrant_client = QdrantClient(
    url="https://8366dca9-9b40-481c-9a7c-102b62b118c2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="Ft3t6L99uoTsJW8IOk5VH2byKU-BQYqiieuxXFBDp99wo75od0ddAw",
)

def create_collection(collection_name, vector_size, distance_metric):
    try:
        # ตรวจสอบว่าคอลเลกชันมีอยู่แล้วหรือไม่
        collections = qdrant_client.get_collections().collections
        existing_collections = [col.name for col in collections]

        if collection_name in existing_collections:
            print(f"Collection {collection_name} already exists.")
        else:
            # สร้างคอลเลกชัน
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=vector_size,  # ขนาดของเวกเตอร์ embedding
                    distance=distance_metric  # ระยะทางที่ใช้ในการคำนวณ
                )
            )
            print(f"Collection {collection_name} created successfully.")
    except Exception as e:
        print(f"An error occurred while creating the collection: {e}")

def image_embedding(path):
    img = Image.open(path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt", size=(640, 640))
    inputs = {k: v.to(device) for k, v in inputs.items()}  # ส่งข้อมูลไปยัง GPU
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    embedding = outputs.cpu().numpy().flatten()
    return embedding

def process_images_in_folder(folder_path, collection_name):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    points_count = len(image_files)  # นับจำนวนภาพที่นำเข้า

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(folder_path, image_file)
        embedding = image_embedding(image_path)
        class_name = os.path.basename(folder_path)  # ใช้ชื่อโฟลเดอร์เป็นคลาส
        point_id = str(uuid.uuid4())  # ใช้ UUID เป็น ID
        point = PointStruct(id=point_id, vector=embedding.tolist(), payload={"class": class_name, "image_label": image_file})
        try:
            qdrant_client.upsert(collection_name=collection_name, points=[point])
            print(f"Uploaded {image_file} to collection {collection_name} with class {class_name}")
        except Exception as e:
            print(f"An error occurred while uploading {image_file}: {e}")

    return points_count  # ส่งกลับจำนวนภาพที่ประมวลผล

def process_all_folders(base_folder_path):
    collection_name = "vector_CLIP"  # ใช้ชื่อคอลเลกชันเดียวสำหรับเก็บข้อมูลทั้งหมด

    create_collection(collection_name, vector_size=768, distance_metric='Cosine')  # ขนาดเวกเตอร์ของ CLIPModel คือ 768

    subfolders = [f.path for f in os.scandir(base_folder_path) if f.is_dir()]

    total_points = 0  # ตัวแปรสำหรับเก็บจำนวนภาพทั้งหมด

    for subfolder in subfolders:
        points_count = process_images_in_folder(subfolder, collection_name)
        total_points += points_count
        print(f"Embedding data from {os.path.basename(subfolder)} has been uploaded to Qdrant. Points count: {points_count}")

    print(f"Total points uploaded: {total_points}")  # แสดงผลรวมของจำนวนภาพที่ประมวลผล

# ตัวอย่างการใช้งาน
base_folder_path = '/content/drive/MyDrive/drug/Corp_4800'
process_all_folders(base_folder_path)

In [ ]:
import os

def count_images_in_folder(folder_path):
    total_images = 0

    # ตรวจสอบว่ามีโฟลเดอร์ย่อยหรือไม่
    for root, dirs, files in os.walk(folder_path):
        # นับไฟล์ภาพในแต่ละโฟลเดอร์
        image_files = [f for f in files if f.endswith(('.png', '.jpg', '.jpeg'))]
        total_images += len(image_files)

        # แสดงชื่อโฟลเดอร์และจำนวนภาพในโฟลเดอร์นั้นๆ
        if root != folder_path:  # ข้ามโฟลเดอร์หลัก
            print(f"Folder: {root}")
            print(f"Number of images: {len(image_files)}")

    print(f"\nTotal number of images: {total_images}")

# ตัวอย่างการใช้งาน
base_folder_path = '/content/drive/MyDrive/drug/Corp_4800'
count_images_in_folder(base_folder_path)

Total number of images: 4765